# مِسٹرال ماڈلز کے ساتھ تعمیر

## تعارف

یہ سبق شامل کرے گا:  
- مختلف مِسٹرال ماڈلز کی تلاش  
- ہر ماڈل کے استعمال کے کیسز اور منظرناموں کو سمجھنا  
- کوڈ کے نمونے جو ہر ماڈل کی منفرد خصوصیات دکھاتے ہیں۔


## مسٹرال ماڈلز

اس سبق میں، ہم 3 مختلف مسٹرال ماڈلز کا جائزہ لیں گے:  
**مسٹرال لارج**، **مسٹرال سمال** اور **مسٹرال نیمو**۔

ان میں سے ہر ماڈل گٹ ہب ماڈل مارکیٹ پلیس پر مفت دستیاب ہے۔ اس نوٹ بک میں کوڈ چلانے کے لیے انہی ماڈلز کا استعمال کیا جائے گا۔ یہاں گٹ ہب ماڈلز کو استعمال کرنے کی مزید تفصیلات ہیں تاکہ آپ [AI ماڈلز کے ساتھ پروٹوٹائپ بنا سکیں](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)۔


## Mistral Large 2 (2407)
Mistral Large 2 اس وقت Mistral کا فلیگ شپ ماڈل ہے اور اسے انٹرپرائز استعمال کے لیے ڈیزائن کیا گیا ہے۔

یہ ماڈل اصل Mistral Large کا اپ گریڈ ہے جو پیش کرتا ہے  
-  بڑا کانٹیکسٹ ونڈو - 128k بمقابلہ 32k  
-  ریاضی اور کوڈنگ کے کاموں میں بہتر کارکردگی - 76.9% اوسط درستگی بمقابلہ 60.4%  
-  کثیر لسانی کارکردگی میں اضافہ - زبانوں میں شامل ہیں: انگریزی، فرانسیسی، جرمن، ہسپانوی، اطالوی، پرتگالی، ڈچ، روسی، چینی، جاپانی، کوریائی، عربی، اور ہندی۔

ان خصوصیات کے ساتھ، Mistral Large میں مہارت حاصل ہے  
- *Retrieval Augmented Generation (RAG)* - بڑے کانٹیکسٹ ونڈو کی وجہ سے  
- *Function Calling* - اس ماڈل میں نیٹیو فنکشن کالنگ ہے جو بیرونی ٹولز اور APIs کے ساتھ انضمام کی اجازت دیتی ہے۔ یہ کالز متوازی طور پر یا ایک کے بعد ایک ترتیب وار کی جا سکتی ہیں۔  
- *Code Generation* - یہ ماڈل Python، Java، TypeScript اور C++ کی جنریشن میں مہارت رکھتا ہے۔


### RAG کی مثال Mistral Large 2 کا استعمال کرتے ہوئے


اس مثال میں، ہم Mistral Large 2 استعمال کر رہے ہیں تاکہ ایک متن دستاویز پر RAG پیٹرن چلایا جا سکے۔ سوال کورین زبان میں لکھا گیا ہے اور مصنف کی کالج سے پہلے کی سرگرمیوں کے بارے میں پوچھتا ہے۔

یہ Cohere Embeddings ماڈل استعمال کرتا ہے تاکہ متن دستاویز اور سوال دونوں کی ایمبیڈنگز بنائی جا سکیں۔ اس نمونے کے لیے، یہ faiss پائتھن پیکیج کو ویکٹر اسٹور کے طور پر استعمال کرتا ہے۔

Mistral ماڈل کو بھیجا گیا پرامپٹ دونوں سوالات اور بازیافت شدہ حصوں کو شامل کرتا ہے جو سوال سے مشابہت رکھتے ہیں۔ پھر ماڈل ایک قدرتی زبان میں جواب فراہم کرتا ہے۔


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small Mistral خاندان کے ماڈلز میں سے ایک اور ماڈل ہے جو premier/enterprise زمرے کے تحت آتا ہے۔ جیسا کہ نام سے ظاہر ہے، یہ ماڈل ایک Small Language Model (SLM) ہے۔ Mistral Small استعمال کرنے کے فوائد یہ ہیں کہ یہ: 
- Mistral LLMs جیسے Mistral Large اور NeMo کے مقابلے میں لاگت کی بچت - 80% قیمت میں کمی
- کم تاخیر - Mistral کے LLMs کے مقابلے میں تیز تر جواب
- لچکدار - مختلف ماحول میں کم وسائل کی پابندیوں کے ساتھ تعینات کیا جا سکتا ہے۔ 


Mistral Small کے لیے بہترین ہے: 
- متن پر مبنی کام جیسے خلاصہ سازی، جذباتی تجزیہ اور ترجمہ۔ 
- ایسی ایپلیکیشنز جہاں بار بار درخواستیں کی جاتی ہیں کیونکہ یہ لاگت کے لحاظ سے مؤثر ہے 
- کم تاخیر والے کوڈ کے کام جیسے جائزہ اور کوڈ کی تجاویز 


## مِسٹرال اسمال اور مِسٹرال لارج کا موازنہ

مِسٹرال اسمال اور لارج کے درمیان تاخیر میں فرق دکھانے کے لیے، نیچے دیے گئے سیلز چلائیں۔

آپ کو 3-5 سیکنڈ کے درمیان ردعمل کے اوقات میں فرق نظر آئے گا۔ اسی پرامپٹ پر ردعمل کی لمبائی اور انداز کو بھی نوٹ کریں۔


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

اس سبق میں زیر بحث دیگر دو ماڈلز کے مقابلے میں، Mistral NeMo واحد مفت ماڈل ہے جس کے ساتھ Apache2 لائسنس ہے۔

اسے Mistral کے پہلے کے اوپن سورس LLM، Mistral 7B، کا اپ گریڈ سمجھا جاتا ہے۔

NeMo ماڈل کی کچھ دیگر خصوصیات یہ ہیں:

- *زیادہ مؤثر ٹوکنائزیشن:* یہ ماڈل عام طور پر استعمال ہونے والے tiktoken کے بجائے Tekken ٹوکنائزر استعمال کرتا ہے۔ اس سے زیادہ زبانوں اور کوڈ پر بہتر کارکردگی ممکن ہوتی ہے۔

- *فائن ٹیوننگ:* بنیادی ماڈل فائن ٹیوننگ کے لیے دستیاب ہے۔ اس سے ایسے استعمال کے معاملات میں زیادہ لچک ملتی ہے جہاں فائن ٹیوننگ کی ضرورت ہو سکتی ہے۔

- *نیٹیو فنکشن کالنگ* - Mistral Large کی طرح، اس ماڈل کو فنکشن کالنگ پر تربیت دی گئی ہے۔ یہ اسے منفرد بناتا ہے کیونکہ یہ اوپن سورس ماڈلز میں سے ایک ہے جو ایسا کرتا ہے۔


## Mistral NeMo

اس سبق میں زیر بحث دیگر دو ماڈلز کے مقابلے میں، Mistral NeMo واحد مفت ماڈل ہے جس کے ساتھ Apache2 لائسنس ہے۔

اسے Mistral کے پہلے کے اوپن سورس LLM، Mistral 7B، کا اپ گریڈ سمجھا جاتا ہے۔

NeMo ماڈل کی کچھ دیگر خصوصیات یہ ہیں:

- *زیادہ مؤثر ٹوکنائزیشن:* یہ ماڈل عام طور پر استعمال ہونے والے tiktoken کے بجائے Tekken ٹوکنائزر استعمال کرتا ہے۔ اس سے زیادہ زبانوں اور کوڈ پر بہتر کارکردگی ممکن ہوتی ہے۔

- *فائن ٹیوننگ:* بنیادی ماڈل فائن ٹیوننگ کے لیے دستیاب ہے۔ اس سے ایسے استعمال کے معاملات میں زیادہ لچک ملتی ہے جہاں فائن ٹیوننگ کی ضرورت ہو سکتی ہے۔

- *نیٹیو فنکشن کالنگ* - Mistral Large کی طرح، اس ماڈل کو فنکشن کالنگ پر تربیت دی گئی ہے۔ یہ اسے منفرد بناتا ہے کیونکہ یہ اوپن سورس ماڈلز میں سے ایک ہے جو ایسا کرتا ہے۔


### ٹوکنائزرز کا موازنہ

اس نمونے میں، ہم دیکھیں گے کہ Mistral NeMo ٹوکنائزیشن کو Mistral Large کے مقابلے میں کیسے ہینڈل کرتا ہے۔

دونوں نمونے ایک ہی پرامپٹ لیتے ہیں لیکن آپ کو دیکھنا چاہیے کہ NeMo Mistral Large کے مقابلے میں کم ٹوکنز واپس کرتا ہے۔


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## سیکھنا یہاں ختم نہیں ہوتا، سفر جاری رکھیں

اس سبق کو مکمل کرنے کے بعد، ہمارے [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) کو دیکھیں تاکہ آپ اپنی Generative AI کی معلومات کو مزید بڑھا سکیں!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**دستخطی دستبرداری**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کے ذریعے ترجمہ کی گئی ہے۔ اگرچہ ہم درستگی کے لیے کوشاں ہیں، براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستیاں ہو سکتی ہیں۔ اصل دستاویز اپنی مادری زبان میں ہی معتبر ماخذ سمجھی جانی چاہیے۔ اہم معلومات کے لیے پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کی ذمہ داری ہم پر عائد نہیں ہوتی۔
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
